In [1]:
import numpy as np
import csv
import json
import os
from collections import defaultdict

Load positive and negative labels for all images.
Options: 'train' or 'validation'

In [65]:
db_type = 'validation'

Get image IDs, then load image-level labels for all the images.

In [66]:
fn_iid = './data/annotations/{}-images-with-rotation.csv'.format(db_type)
if not os.path.isfile(fn_iid):
    fn_iid = './data/annotations/{}-images-boxable-with-rotation.csv'.format(db_type)

iid_all = []
with open(fn_iid, 'r', newline='') as fh:
    reader = csv.reader(fh)
    for ii, row in enumerate(reader):
        if ii == 0:
            continue
        iid_all.append(row[0])
iid_all = set(iid_all)

In [67]:
fn_image_label = './data/annotations/{}-annotations-human-imagelabels.csv'.format(db_type)
pos_labels = defaultdict(list)
neg_labels = defaultdict(list)
pos_counter = defaultdict(lambda: 0)
neg_counter = defaultdict(lambda: 0)

with open(fn_image_label, 'r', newline='') as fh:
    reader = csv.reader(fh)
    for ii, row in enumerate(reader):
        if ii == 0:
            continue
        iid, _, cid, is_pos = row
        if iid not in iid_all:
            continue
#         cname = cls_desc[cid]
        if int(is_pos) == 1:
            pos_labels[iid].append(cid)
            pos_counter[cid] += 1
        else:
            neg_labels[iid].append(cid)
            neg_counter[cid] += 1
            
pos_labels = dict(pos_labels)
neg_labels = dict(neg_labels)

If train, get valid labels.

In [68]:
if db_type == 'train':
    valid_pos = [k for k, v in pos_counter.items() if v >= 200]
    valid_neg = [k for k, v in neg_counter.items() if v >= 200]
    valid_pn = set(valid_pos + valid_neg)
    valid_all = set([k for k in valid_pn if pos_counter[k] + neg_counter[k] >= 500])
    
    fn_class_desc = 'data/annotations/class-descriptions.csv'
    class_desc = {}
    class_desc = open(fn_class_desc, 'r').read().splitlines()
    class_desc = [cdesc.split(',') for cdesc in class_desc]
    class_desc = {c[0].strip(): c[1].strip() for c in class_desc}    
        
    # save valid labels in a file
    fn_valid_labels = 'data/annotations/hyunjoon/labels_used.txt'
    with open(fn_valid_labels, 'w') as fh:
        for k in valid_all:
            np = pos_counter[k]
            nn = neg_counter[k]
            na = np + nn
            fh.write('{}, {}, {}, {}, {}\n'.format(k, np, nn, na, class_desc[k]))
else:
    valid_all = open('data/annotations/hyunjoon/labels_used.txt').read().splitlines()
    valid_all = [v.split(',')[0].strip() for v in valid_all]

In [69]:
len(valid_all)

1573

Save labels into the two files, one for positive and one for negative

In [70]:
iid_all = []

In [71]:
fn_positive_labels = 'data/annotations/hyunjoon/{}_positive_imagelabel.txt'.format(db_type)
with open(fn_positive_labels, 'w') as fh:
    for k, v in pos_labels.items():
        vv = [vi for vi in v if vi in valid_all]
        if not vv:
            continue
        # -1 to name2idx to remove 'entity'
        one_line = '{}, '.format(k) + ', '.join([vi for vi in vv])
        fh.write(one_line + '\n')
        iid_all.append(k)

In [72]:
fn_negative_labels = 'data/annotations/hyunjoon/{}_negative_imagelabel.txt'.format(db_type)
with open(fn_negative_labels, 'w') as fh:
    for k, v in neg_labels.items():
        vv = [vi for vi in v if vi in valid_all]
        if not vv:
            continue
        # -1 to name2idx to remove 'entity'
        one_line = '{}, '.format(k) + ', '.join([vi for vi in vv])
        fh.write(one_line + '\n')
        iid_all.append(k)

In [73]:
iid_all = set(iid_all)
fn_imagelist = 'data/annotations/hyunjoon/{}_imagelist.txt'.format(db_type)
with open(fn_imagelist, 'w') as fh:
    for iid in iid_all:
        fh.write(iid + '\n')